* 역자 : Data ScienceTutorial for Beginners의 시리즈중 시각화 파트입니다.
        저번엔 Seaborn을 이용한 시각화였다면 이번엔 Bokeh을 이용한 시각화입니다.

# 소개
우리는 이 튜토리얼에서 bokeh 라이브러리의 기초적인 부분을 학습할 수 있다.
Bokeh 라이브러리는 상호 시각화 라이브러리입니다.
<br> 나(원글쓴이)는 bokeh 튜토리얼을 2부분으로 나누었다.
왜냐하면 캐글에서 bokeh을 실행 시킬경우 browser와 충돌이 있다.

1. PART 1:
    1. Basic Data Exploration with Pandas
    1. Explanation of Bokeh Packages
    1. Plotting with Glyphs
    1. Additional Glyps
    1. Data Formats
    1. Customizing Glyphs
    1. Layouts
    1. Linking Plots
1. PART 2: https://www.kaggle.com/kanncaa1/interactive-bokeh-tutorial-part-2/editnb
    1. Callbacks 
        * Slider
        * dropdowns


In [2]:
# 이 파이썬3 환경은 유용한 분석 라이브러리들이 있다
# kaggle/python docker image는 다음 링크에서 받을 수 있다.: https://github.com/kaggle/docker-python
# 예를 들어 몇몇 유용한 패키지를 로드해보자.

import numpy as np # linear algebra
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv)

# 입력 데이터 파일은 "../input/" 폴더에 있다.
# 예를 들어 실행을 클릭하거나 Shift+Enter를 누르면 입력 폴더에 있는 파일이 나열된다.

file_path = "../../DataSet/Video Game Sales"

from subprocess import check_output
# print(check_output(["ls", file_path]).decode("utf8"))

# 현재 디렉토리에 쓰는 모든 결과는 출력으로 저장된다.

##  Pandas로 Data 기초 살펴보기
<br> 모든 작업을 하기전에 우리는 read_csv()를 이용하여 데이터를 가져와야한다.
<br> 시각화를 위해서는 데이터를 이해하고 데이터에 nan값이 있는지 확인해야 합니다.

<br> 우리는 pandas library를 사용할겁니다.

    * read_csv()
    * info()
    * head()

In [4]:
# info()함수에서 볼 수 있다시피 16598이 있다.
# 그러나, 연도에는 16327개의 항목이 있습니다. 이것은 연도에 NAN 값이 있다는것을 의미합니다.
# 또한 연도는 integer형이여야 하지만, 이것은 float형으로 주어진다. 그러므로 우리가 이것을 integer형으로 변환해주어야 한다.
# 게시자(publisher)는 NAN 값을 가지고 있다.
# Also Year should be integer but it is given as float. Therefore we will convert it.
# In addition, publisher has NAN values.
data = pd.read_csv(file_path+"/vgsales.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [5]:
# Nan값을 없애보자.
data.dropna(how="any", inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16291 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16291 non-null  int64  
 1   Name          16291 non-null  object 
 2   Platform      16291 non-null  object 
 3   Year          16291 non-null  float64
 4   Genre         16291 non-null  object 
 5   Publisher     16291 non-null  object 
 6   NA_Sales      16291 non-null  float64
 7   EU_Sales      16291 non-null  float64
 8   JP_Sales      16291 non-null  float64
 9   Other_Sales   16291 non-null  float64
 10  Global_Sales  16291 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.5+ MB


In [6]:
# float형을 int형으로 변환해보자.
data.Year = data.Year.astype(int)
data.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


## Bokeh Packages 설명
bokeh 라이브러리의 경우 몇가지를 사용합니다 :
* output_file: that save our figure with .html extension
* show: show the figure
* figure: creates empty figure
* ColumnarDataSource: Data source of bokeh
* HoverTool: like cursor
* CategoricalColorMapper: Like a hue in seaborn. If you do not know it look at my seaborn tutorial
    * https://www.kaggle.com/kanncaa1/seaborn-for-beginners
* Row and column: puts plots in row order or column order in figure
* gridplot
* Tabs and Panel: Panel is figure for each plot and tab is like button
    
